In [206]:

import numpy as np

data = np.load('HW4_data.npy')
data[:,2]
length = len(data[:,0])
train = data[:50000,:]
test = data[-10000:,:]

In [0]:
########## (a) Manually Setting up MLP ##########
dic = {}

### Manually Selected Weights
for i in range(10):
    dic[i] = [0]*2
dic[0][0] = np.array([-1,1])
dic[0][1] = -5
dic[1][0] = np.array([1,2])
dic[1][1] = 14
dic[2][0] = np.array([1,0])
dic[2][1] = 6
dic[3][0] = np.array([1,-2])
dic[3][1] = -6
dic[4][0] = np.array([-1,-1])
dic[4][1] = -15
dic[5][0] = np.array([-1,0.5])
dic[5][1] = -3.5
dic[6][0] = np.array([1,1])
dic[6][1] = 5
dic[7][0] = np.array([1,0])
dic[7][1] = 2
dic[8][0] = np.array([1,-1])
dic[8][1] = -5
dic[9][0] = np.array([-1,-0.5])
dic[9][1] = -8.5
for i in range(10):
    dic[i][1] *= 100

### Use Manual Model to make predictions
ans = [0]*length
correct = 0
for i in range(length):
    x = np.array([data[i][1],data[i][0]])
    y1 = 0
    y2 = 0
    for j in range(5):
        y1 += 1*( x.dot(dic[j][0]) - dic[j][1] > 0 )
        y2 += 1*( x.dot(dic[j+5][0]) - dic[j+5][1] > 0 )
    if y1 >= 5 or y2 >= 5:
        ans[i] = 1
    if data[i,2] == ans[i]:
        correct+= 1
print(correct)

60000


In [0]:
########## (b) ##########
def sigmoid(f1):
    sig = 1/( 1+np.exp(-f1) )
    return sig
for t in range(5):
    print("Train and Test with random initialization round " + str(t) + ":")

    ### Weights and bias Initialization
    w1 = np.random.uniform(-2,2,(10,2))
    b1 = np.random.uniform(-1500, 1400, (10,1))

    w2 = np.full((2,10), 1.0)
    b2 = np.full((2,1), 0.0)

    w3 = np.full((1,2), 1.0)
    b3= 0

    ## Number of iterations and stepsize hyperparameter
    iter_n = 100
    step = 0.1
    decay_counter = 0
    T_Loss = 0
    ## Training
    for p in range(iter_n):
        decay_counter += 1
        if decay_counter == 20:
            step*=0.1
            decay_counter = 0
        correct = 0
        for i in range(50000):
            x = np.array([data[i][0],data[i][1]])
            x = x.reshape(2,1)

            f1 = w1.dot(x) + b1 #1x10
            sigf1 = sigmoid(f1) #1x10

            f2 = w2.dot(sigf1) + b2 #1x2
            sigf2 = sigmoid(f2) #1x2

            f3 = w3.dot(sigf2) + b3 # C
            sigf3 = sigmoid(f3)
            y_hat = 1*(sigf3>0.5)

            ## Gradient of w3, b3
            dsigf3 = -(data[i,2]/sigf3) + (1-data[i,2])/(1-sigf3) # C
            df3 = dsigf3 * ( sigf3 * (1-sigf3) ) # C
            dw3 = df3 * sigf2.T #1x2
            db3 = df3 # C

            ## Gradient of w2, b2
            dsigf2 = df3 * w3 #1x2
            df2 = dsigf2.T * ( sigf2 * (1-sigf2) ) #1x2
            dw2 = df2.dot(sigf1.T) #2x10
            db2 = df2 #1x2

            ## Gradient of w1, b1
            dsigf1 = df2.T.dot(w2) #1x10
            df1 = dsigf1.T * ( sigf1 * (1-sigf1) ) #10x1
            dw1 = df1.dot(x.T) #10x2
            db1 = df1

            ## BP
            w1 -= step*dw1
            b1 -= step*db1

            w2 -= step*dw2
            b2 -= step*db2

            w3 -= step*dw3
            b3 -= step*db3

            if p == iter_n - 1:
                y = data[i,2]
                T_Loss += -( y*np.log(sigf3) + (1-y)*np.log(1-sigf3) )
                if y_hat == data[i,2]:
                    correct+=1
    print("\tAverage Loss is: " + str(T_Loss/10000))
    print("\tTrain Accuracy: " + str(correct/50000*100) + "%")

    ### Testing
    test_correct = 0
    Loss = 0
    for i in range(len(test)):
        x = np.array([test[i][0],test[i][1]])
        x = x.reshape(2,1)

        f1 = w1.dot(x) + b1 #1x10
        sigf1 = sigmoid(f1) #1x10

        f2 = w2.dot(sigf1) + b2 #1x2
        sigf2 = sigmoid(f2) #1x2

        f3 = w3.dot(sigf2) + b3 # C
        sigf3 = sigmoid(f3)
        y_hat = 1*(sigf3>0.5)
        
        y = test[i,2]
        Loss += -( y*np.log(sigf3) + (1-y)*np.log(1-sigf3) )
        if y_hat == test[i,2]:
            test_correct+=1
    print("\tAverage Loss is: " + str(Loss/10000))
    print("\tTest Accuracy: " + str(test_correct/10000*100) + "%")


Train and Test with random initialization round 0:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


	Average Loss is: [[2.33107402]]
	Train Accuracy: 82.282%
	Average Loss is: [[0.65754397]]
	Test Accuracy: 69.92%
Train and Test with random initialization round 1:
	Average Loss is: [[2.33549455]]
	Train Accuracy: 82.282%
	Average Loss is: [[0.65702699]]
	Test Accuracy: 69.92%
Train and Test with random initialization round 2:
	Average Loss is: [[2.0577669]]
	Train Accuracy: 82.282%
	Average Loss is: [[0.59243162]]
	Test Accuracy: 69.92%
Train and Test with random initialization round 3:
	Average Loss is: [[2.33283494]]
	Train Accuracy: 82.282%
	Average Loss is: [[0.65645398]]
	Test Accuracy: 69.92%
Train and Test with random initialization round 4:
	Average Loss is: [[2.33261661]]
	Train Accuracy: 82.282%
	Average Loss is: [[0.65657701]]
	Test Accuracy: 69.92%


In [0]:
######## (C) Increased Width from 10 to 15 ########

def sigmoid(f1):
    sig = 1/( 1+np.exp(-f1) )
    return sig

### Weights and bias Initialization
w1 = np.random.uniform(-2,2,(15,2))
b1 = np.random.uniform(-1500, 1400, (15,1))

w2 = np.full((2,15), 1.0)
b2 = np.full((2,1), 0.0)

w3 = np.full((1,2), 1.0)
b3= 0

## Number of iterations and stepsize hyperparameter
iter_n = 100
step = 0.1
decay_counter = 0
## Training
for p in range(iter_n):
    decay_counter += 1
    if decay_counter == 20:
        step*=0.1
        decay_counter = 0
    correct = 0
    for i in range(50000):
        x = np.array([data[i][0],data[i][1]])
        x = x.reshape(2,1)

        f1 = w1.dot(x) + b1 #1x10
        sigf1 = sigmoid(f1) #1x10

        f2 = w2.dot(sigf1) + b2 #1x2
        sigf2 = sigmoid(f2) #1x2

        f3 = w3.dot(sigf2) + b3 # C
        sigf3 = sigmoid(f3)
        y_hat = 1*(sigf3>0.5)

        ## Gradient of w3, b3
        dsigf3 = -(data[i,2]/sigf3) + (1-data[i,2])/(1-sigf3) # C
        df3 = dsigf3 * ( sigf3 * (1-sigf3) ) # C
        dw3 = df3 * sigf2.T #1x2
        db3 = df3 # C

        ## Gradient of w2, b2
        dsigf2 = df3 * w3 #1x2
        df2 = dsigf2.T * ( sigf2 * (1-sigf2) ) #1x2
        dw2 = df2.dot(sigf1.T) #2x10
        db2 = df2 #1x2

        ## Gradient of w1, b1
        dsigf1 = df2.T.dot(w2) #1x10
        df1 = dsigf1.T * ( sigf1 * (1-sigf1) ) #10x1
        dw1 = df1.dot(x.T) #10x2
        db1 = df1

        ## BP
        w1 -= step*dw1
        b1 -= step*db1

        w2 -= step*dw2
        b2 -= step*db2

        w3 -= step*dw3
        b3 -= step*db3

        if p == iter_n - 1:
            if y_hat == data[i,2]:
                correct+=1
print("\tTrain Accuracy: " + str(correct/50000*100) + "%")

### Testing
Loss = 0
test_correct = 0
for i in range(len(test)):
    x = np.array([test[i][0],test[i][1]])
    x = x.reshape(2,1)

    f1 = w1.dot(x) + b1 #1x10
    sigf1 = sigmoid(f1) #1x10

    f2 = w2.dot(sigf1) + b2 #1x2
    sigf2 = sigmoid(f2) #1x2

    f3 = w3.dot(sigf2) + b3 # C
    sigf3 = sigmoid(f3)
    y_hat = 1*(sigf3>0.5)

    if y_hat == test[i,2]:
        test_correct+=1
    y = test[i,2]
    Loss += -( y*np.log(sigf3) + (1-y)*np.log(1-sigf3) )
print("\tAverage Loss is: " + str(Loss/10000))
print("\tTest Accuracy: " + str(test_correct/10000*100) + "%")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


	Train Accuracy: 82.282%
	Average Loss is: [[0.65701566]]
	Test Accuracy: 69.92%


In [0]:
######## Increased Depth with an extra layer ########

def sigmoid(f1):
    sig = 1/( 1+np.exp(-f1) )
    return sig

### Weights and bias Initialization
w1 = np.random.uniform(-2,2,(10,2))
b1 = np.random.uniform(-1500, 1400, (10,1))

w2 = np.full((5,10), 1.0)
b2 = np.full((5,1), 0.0)

wplus = np.full((2,5), 1.0)
bplus = np.full((2,1), 0.0)

w3 = np.full((1,2), 1.0)
b3 = 0

## Number of iterations and stepsize hyperparameter
iter_n = 100
step = 100
decay_counter = 0
## Training
for p in range(iter_n):
    decay_counter += 1
    if decay_counter == 20:
        step*=0.1
        decay_counter = 0
    correct = 0
    for i in range(50000):
        x = np.array([data[i][0],data[i][1]])
        x = x.reshape(2,1)

        f1 = w1.dot(x) + b1 #1x10
        sigf1 = sigmoid(f1) #1x10

        f2 = w2.dot(sigf1) + b2 #1x2
        sigf2 = sigmoid(f2) #1x2
        
        fplus = wplus.dot(sigf2) + bplus #2x1
        sigplus = sigmoid(fplus) #2x1

        f3 = w3.dot(sigplus) + b3 # C
        sigf3 = sigmoid(f3)
        y_hat = 1*(sigf3>0.5)

        ## Gradient of w3, b3
        dsigf3 = -(data[i,2]/sigf3) + (1-data[i,2])/(1-sigf3) # C
        df3 = dsigf3 * ( sigf3 * (1-sigf3) ) # C
        dw3 = df3 * sigplus.T #1x2
        db3 = df3 # C

        ## Gradient of wplus, bplus
        dsigfplus = df3 * w3 #1x2
        dfplus = dsigfplus.T * ( sigplus * (1-sigplus) ) #2x1
        dwplus = dfplus.dot(sigf2.T) #2x2
        dbplus = dfplus #2x1
        
        ## Gradient of w2, b2
        dsigf2 = dfplus.T.dot(wplus) #1x2
        df2 = dsigf2.T * ( sigf2 * (1-sigf2) ) #1x2
        dw2 = df2.dot(sigf1.T) #2x10
        db2 = df2 #1x2
        
        ## Gradient of w1, b1
        dsigf1 = df2.T.dot(w2) #1x10
        df1 = dsigf1.T * ( sigf1 * (1-sigf1) ) #10x1
        dw1 = df1.dot(x.T) #10x2
        db1 = df1

        ## BP
        w1 -= step*dw1
        b1 -= step*db1

        w2 -= step*dw2
        b2 -= step*db2
        
        wplus -= step*dwplus
        bplus -= step*dbplus
        
        w3 -= step*dw3
        b3 -= step*db3

        if p == iter_n - 1:
            if y_hat == data[i,2]:
                correct+=1
print("\tTrain Accuracy: " + str(correct/50000*100) + "%")

### Testing
test_correct = 0
Loss = 0
for i in range(len(test)):
    x = np.array([test[i][0],test[i][1]])
    x = x.reshape(2,1)

    f1 = w1.dot(x) + b1 #1x10
    sigf1 = sigmoid(f1) #1x10

    f2 = w2.dot(sigf1) + b2 #1x2
    sigf2 = sigmoid(f2) #1x2

    fplus = wplus.dot(sigf2) + bplus #2x1
    sigplus = sigmoid(fplus) #2x1

    f3 = w3.dot(sigplus) + b3 # C
    sigf3 = sigmoid(f3)
    y_hat = 1*(sigf3>0.5)

    if y_hat == test[i,2]:
        test_correct+=1
    y = test[i,2]
    Loss += -( y*np.log(sigf3) + (1-y)*np.log(1-sigf3) )
print("\tAverage Loss is: " + str(Loss/10000))
print("\tTest Accuracy: " + str(test_correct/10000*100) + "%")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


	Train Accuracy: 82.282%
	Average Loss is: [[0.6581213]]
	Test Accuracy: 69.92%
